In [1]:
# 최신순으로 리뷰 내용 50건씩 크롤링
# 크롬 108.xx 버전

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException

import time

In [3]:
def findCafe(phone, name):
    url = f'https://m.map.naver.com/search2/searchMore.naver?query={phone}&page=1'
    response = requests.get(url)
    places = response.json()['result']['site']['list']
    for place in places:
        if place['name'] == name:
            return place['id'][1:]

In [4]:
cafeNames = [
    '헤이안',
    '헤이메르',
    '간절곶 카페루아',
    '카페하든',
    '카페파도',
    '보사노바 커피로스터스 삼척점',
    '뺑드엘리시안',
    '서귀피안 베이커리',
    'PNB 풍년제과 본점',
    'Auspice coffee'
]

cafeNums = [
    '054-252-0200', # 헤이안
    '052-238-0333', # 헤이메르
    '0507-1319-0965', # 간절곶 카페루아
    '0507-1306-7034', # 카페하든
    '0507-1334-3118', # 카페파도
    '033-573-0338', # 보사노바 커피로스터스 삼척점
    '0507-1388-2622', # 뺑드엘리시안
    '0507-1338-8378', # 서귀피안 베이커리
    '0507-1426-6666', # PNB 풍년제과 본점
    '0507-1305-0628' # Auspice coffee
]

cafeIDs = []

In [5]:
for cafeName, cafeNum in zip(cafeNames, cafeNums):
    cafeID = findCafe(cafeNum, cafeName)
    cafeIDs.append(cafeID)
    time.sleep(5)

In [6]:
def clickMore(driver, count=5):
    i = 0

    while i < count: 
            try: 
                driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
                time.sleep(1)
                driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_UP)
                time.sleep(1) 
            
                driver.find_element(By.CSS_SELECTOR, '.lfH3O > a').click()
                time.sleep(2)

                i +=1
                
            except NoSuchElementException: 
                print('-더보기 버튼 모두 클릭 완료-') 
                break 

In [7]:
def seeDetails(driver, action):
    elements = driver.find_elements(By.CSS_SELECTOR, '.YeINN')

    for element in elements:
        more = element.find_elements(By.CSS_SELECTOR, '.ZZ4OK > a > .rvCSr')
        if len(more) > 0:
            action.move_to_element(element).perform()
            time.sleep(1)
            more[0].click()

In [8]:
def scrapReviews(html, cafename, cafelist, reviewlist):
    dom = BeautifulSoup(html, 'lxml')

    reviews = dom.select('.YeINN > .ZZ4OK > a > span:nth-child(1)')
    
    try: 
        for review in reviews: 
            
            text = review.text
        
            cafelist.append(cafename)
            reviewlist.append(text)

    # 리뷰가 없는 경우        
    except NoSuchElementException: 
        print("네이버 리뷰 없음" )

In [9]:
options = webdriver.ChromeOptions()
options.add_argument('headless')

# driver = webdriver.Chrome(options=options)
driver = webdriver.Chrome()
action = ActionChains(driver)

In [10]:
cafes = []
re = []

for cafeID, cafeName in zip(cafeIDs, cafeNames):
    finalurl = f'https://m.place.naver.com/restaurant/{cafeID}/review/visitor?reviewSort=recent'
    driver.get(finalurl)

    clickMore(driver)
    time.sleep(2)

    seeDetails(driver, action)

    html = driver.page_source
    
    scrapReviews(html, cafeName, cafes, re)
    print(f'{cafeName} 리뷰 수집 완료')
    time.sleep(3)

driver.quit()

헤이안 리뷰 수집 완료
헤이메르 리뷰 수집 완료
간절곶 카페루아 리뷰 수집 완료
카페하든 리뷰 수집 완료
카페파도 리뷰 수집 완료
보사노바 커피로스터스 삼척점 리뷰 수집 완료
뺑드엘리시안 리뷰 수집 완료
서귀피안 베이커리 리뷰 수집 완료
PNB 풍년제과 본점 리뷰 수집 완료
Auspice coffee 리뷰 수집 완료


In [11]:
df = pd.DataFrame({'cafeName':cafes,
                   'review': re
                   })

In [12]:
df.head()

,cafeName,review
0,헤이안,바다가 아ㅍ에 보이니깐 너무 조으네요.\n빵도 맛있고 커피도 맛있어요~
1,헤이안,뷰가 정말 멋져요
2,헤이안,오션뷰 너무 예쁘고\n카페 분위기도 취향저격이에요☺️\n인생샷 찍고 왔어요🫰🏻
3,헤이안,맛집이에유
4,헤이안,좋아요


In [13]:
df.tail()

,cafeName,review
548,Auspice coffee,핫한곳
549,Auspice coffee,뷰좋고 분위기좋고 화장실 깨끗하고 커피맛도 나쁘지 않았습니다.
550,Auspice coffee,좋아요
551,Auspice coffee,굿
552,Auspice coffee,오션뷰 카페


In [14]:
df.to_csv('./data/train_data_2.csv', index=False, encoding='utf-8-sig')

In [19]:
t1 = pd.read_csv('./data/train_data_1.csv', encoding='utf-8-sig')
t2 = df.drop('cafeName', axis=1)

train = pd.concat([t1, t2], axis=0)

In [20]:
train.head()

,review
0,맛있어요~
1,만족도 최상의 커피맛집
2,찐 커피맛집ㅎㅎ
3,로마노 진짜 달달하고 맛있고 아이스크림라떼도 꼬소하고 달달하고 진짜 맛있어요!! 마...
4,커조남 커피는 힐링입니다 소금빵도 맛있어요


In [21]:
train.to_csv('./data/train_data.csv', index=False, encoding='utf-8-sig')